# Unitary ansatz entering the VQE

The final energy output of a VQE calculation will crucially depend on the ansatz/form of the parameterized unitary $\hat U(\boldsymbol{\theta})$ employed in state preparation. Here we review two popular approaches, the unitary coupled cluster and qubit coupled cluster methodologies, and benchmark them for energy calculations of small molecules.

In [1]:
import numpy as np
import tequila as tq
from utility import *
threshold = 1e-6 #Cutoff for UCC MP2 amplitudes and QCC ranking gradients

## Unitary Coupled Cluster (UCC)

The UCC ansatz is obtained by 'unitarizing' the traditional coupled cluster ansatz,
$$ e^{\hat T} \rightarrow e^{\hat T - \hat T^\dagger} \equiv \hat U_{\text{UCC}}$$


Due to non-commutativity of terms in $\hat T - \hat T^\dagger$, the UCC ansatz does not have a straightforward decomposition in terms of circuit primitives implementable on the quantum computer. Therefore, to obtain a form which can be compiled, we employ the Trotter approximation. The accuracy of the circuit ansatz relative to the exact UCC operator will be dependent on how many Trotter steps are employed. The number of Trotter steps is commonly set to its minimal value of one to avoid excessive circuit depth.

In [2]:
trotter_steps = 1

### LiH in STO-3G basis

Below is a sample VQE simulation using the UCCSD ansatz compiled using a single trotter step for LiH in minimal basis at $R=2.5$ (Angstrom). For comparison, we can run FCI to obtain the true ground state energy.


In [9]:
xyz_data = get_molecular_data('lih', geometry=1.5, xyz_format=True)
basis='sto-3g'

lih = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis)

print('Number of spin-orbitals (qubits): {} \n'.format(2*lih.n_orbitals))

E_FCI = lih.compute_energy(method='fci', backend='qulacs_gpu')

print('FCI energy: {}'.format(E_FCI))

Number of spin-orbitals (qubits): 12 

FCI energy: -7.882362286810955


The UCCSD VQE optimization is of the form
$$E = \min_{\boldsymbol{\theta}} \langle \text{HF} | \hat U_{\text{UCC}}^\dagger(\boldsymbol{\theta}) \hat H  \hat U_{\text{UCC}} (\boldsymbol{\theta}) | \text{HF} \rangle $$
To expedite the optimization process, we can set the initial guess of the amplitudes to zero, i.e. the optimization will begin at the Hartree Fock state. This heuristic is best suited for when Hartree Fock is believed to be in qualitative agreement with the true ground state. To further alleviate quantum resources, we can estimate the amplitudes using classical electronic structure methods (here, MP2 perturbation theory), and only include the unitaries with non-zero estimated amplitudes.

In [11]:
H = lih.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

U_UCCSD = lih.make_uccsd_ansatz(initial_amplitudes='MP2',threshold=threshold, trotter_steps=trotter_steps)

E = tq.ExpectationValue(H=H, U=U_UCCSD)

print('\nNumber of UCCSD amplitudes: {} \n'.format(len(E.extract_variables())))

print('\nStarting optimization:\n')

result = tq.minimize(objective=E, initial_values={k:0.0 for k in E.extract_variables()}, tol=1e-6, backend='qulacs', method='bfgs')
print('\nObtained UCCSD energy: {}'.format(result.energy))


Hamiltonian has 631 terms


Number of UCCSD amplitudes: 24 


Starting optimization:



KeyboardInterrupt: 

We see that the converged UCCSD energy is in exact agreement with the FCI energy, as expected for a $2$-electron system. 

### H4 in 6-31G basis

Now let us try a larger problem of H$_4$ in 6-31G basis. However, we will restrict the active space. The unrestricted problem leads to a $14$-qubit Hamiltonian, and $34$ UCCSD amplitudes to optimize even after neglecting the zero MP2 amplitudes. Therefore, we will remove some orbital degrees of freedom which are less important in accurately describing the electronic structure. By freezing all orbitals other than $0b_1$, $1b_1$, $2a_1$, and $3a_1$, we reduce the problem to an $8$-qubit Hamiltonian with $8$ UCCSD variational amplitudes.

In [6]:
xyz_data = get_molecular_data('h4', geometry=90, xyz_format=True)

basis = '6-31g'
active = { 'AG':[0,1], 'B1U': [0], 'B2U': [0]}
h4 = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set = basis, active_orbitals=active)

print(list(map(lambda item: item[1][0], h4.orbitals_by_irrep.items())))
print(list(map(lambda item: item[1][0].energy, h4.orbitals_by_irrep.items())))

print('Number of spin-orbitals (qubits): {} \n'.format(2*h4.n_orbitals))

E_FCI = h4.compute_energy(method='fci', backend='qulacs')

print('FCI energy: {}'.format(E_FCI))

There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.
[QuantumChemistryPsi4.OrbitalData(irrep='AG', idx_irrep=0, idx_total=0, energy=-0.6577756529605916), QuantumChemistryPsi4.OrbitalData(irrep='B1U', idx_irrep=0, idx_total=1, energy=-0.3241801225581642), QuantumChemistryPsi4.OrbitalData(irrep='B2U', idx_irrep=0, idx_total=2, energy=-0.021997174708868056), QuantumChemistryPsi4.OrbitalData(irrep='B3G', idx_irrep=0, idx_total=3, energy=0.2489349600035358)]
[-0.6577756529605916, -0.3241801225581642, -0.021997174708868056, 0.2489349600035358]
Number of spin-orbitals (qubits): 8 

There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for fci.
FCI energy: -1.9783982153570014


We will then run the UCCSD VQE simulation (***warning: tq.minimize will take several minutes - 1 hour + to converge for a VQE instance of this size.*** Smaller active spaces can be employed to lower VQE simulation runtimes).

In [7]:
H = h4.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

U_UCCSD = h4.make_uccsd_ansatz(initial_amplitudes='MP2',threshold=threshold, trotter_steps=trotter_steps)

E = tq.ExpectationValue(H=H, U=U_UCCSD)

print('\nNumber of UCCSD amplitudes: {} \n'.format(len(E.extract_variables())))

print('\nStarting optimization:\n')

result = tq.minimize(objective=E,  initial_values={k:0.0 for k in E.extract_variables()}, tol=1e-4, method='bfgs')

print('\nObtained UCCSD energy: {}'.format(result.energy))


Hamiltonian has 105 terms


Number of UCCSD amplitudes: 4 


Starting optimization:

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 128 expectationvalues

active variables : 4

E=-1.93303733  angles= {(3, 0, 3, 0): 0.0, (2, 1, 2, 1): 0.0, (3, 1, 3, 1): 0.0, (2, 0, 2, 0): 0.0}  samples= None
E=-1.92926868  angles= {(3, 0, 3, 0): -0.0867493599653244, (2, 1, 2, 1): -0.09617340564727783, (3, 1, 3, 1): -0.06707680225372314, (2, 0, 2, 0): -0.20908880233764648}  samples= None
E=-1.94840960  angles= {(3, 0, 3, 0): -0.04088724847850404, (2, 1, 2, 1): -0.04532904836757326, (3, 1, 3, 1): -0.031615056087880414, (2, 0, 2, 0): -0.09854903619657156}  samples= None
E=-1.93634995  angles= {(3, 0, 3, 0): 0.04710623158006605, (2, 1, 2, 1): -0.15248204298221218, (3, 1, 3, 1): 0.003034757883097469, (2, 0, 2, 0

In [22]:
print(result)

GPyOptResults(energy=-0.927276119007125, history=OptimizerHistory(energies=array([ 2.57738401e-01, -5.58349949e-01,  4.16314332e-01, -7.20595122e-02,
        4.97766830e-02,  3.63986893e-01,  4.41470282e-01, -4.27460046e-01,
       -3.34552874e-01, -6.88757751e-01,  3.17612575e-02,  6.29450341e-01,
        5.73832760e-01,  3.54992200e-01, -5.21438092e-01,  2.77093020e-01,
        2.55119553e-02, -2.96691072e-01, -5.85835661e-01,  4.25659865e-01,
        2.43356886e-01,  4.87073315e-01,  1.83683410e-01, -2.86054791e-01,
       -1.07986374e-01,  4.28583672e-01, -1.77113055e-01,  2.07521067e-01,
       -4.13718844e-02, -3.19489259e-01,  3.47232406e-01,  1.92401895e-01,
       -3.89797152e-01,  1.52644003e-01, -3.07677173e-01,  4.36022308e-01,
        2.24658607e-01, -2.70690690e-02, -7.84635290e-01, -2.33506975e-01,
        2.10487341e-01,  2.76904295e-01, -6.02145590e-01,  2.39555745e-01,
        3.90147986e-01,  3.21791107e-01, -5.16908220e-01, -5.44215014e-01,
       -3.99239246e-01,  

We obtain a UCCSD energy within chemical accuracy (< 1.6 miliHartree from FCI energy) for the H$_2$O molecule near equilibrium geometry. As the bond distance approaches the dissociation limit, the energy deviation from FCI is typically expected to increase as electronic correlations increase during covalent bond-breaking. Furthermore, as HF becomes energetically more distant from the FCI wavefunction, the initial guess of all amplitudes being zero may lead to a local minimum. One can then instead initialize the amplitudes using random guesses, and repeat for $n$ samples to attempt to find the global minimum. To facilitate this, one can run the commented lines below. This will execute $n$ instances of tq.minimize for uniformly random initial parameters in interval $[0, 2\pi]$ and return the lowest optimization result.

In [ ]:
n=10
result = minimize_E_random_guesses(objective=E, method='BFGS', tol=1e-4, n=n)

## Qubit Coupled Cluster (QCC)

In contrast to UCC, the QCC methodology makes no direct reference to fermionic algebra and seeks to construct an efficient ansatz directly in qubit-space by finding multi-qubit Pauli strings (entanglers) which lower energy. This is done through an energy-lowering heuristic employing the energy gradient with respect to a Pauli strings variational amplitude. As opposed to UCCSD, the circuit depth and number of parameter is chosen to meet hardware limitations, i.e. one must choose how many exponentiated Pauli strings will be entering the QCC ansatz.

### LiH in STO-3G basis

Below we perform the entangler screening protocol for LiH in minimal basis, and obtain one grouping of entanglers with non-zero energy gradient. We then select one of them to be used in the QCC VQE simulation.

In [26]:
xyz_data = get_molecular_data('lih', geometry=1.5, xyz_format=True)
basis='sto-3g'

lih = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set='sto-3g')

hf_reference = hf_occ(2*lih.n_orbitals, lih.n_electrons)

H = lih.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

#Define number of entanglers to enter ansatz
n_ents = 1

#Rank entanglers using energy gradient criterion
ranked_entangler_groupings = generate_QCC_gradient_groupings(H.to_openfermion(), 
                                                             2*lih.n_orbitals, 
                                                             hf_reference, 
                                                             cutoff=threshold)

print('Grouping gradient magnitudes (Grouping : Gradient magnitude):')
for i in range(len(ranked_entangler_groupings)):
    print('{} : {}'.format(i+1,ranked_entangler_groupings[i][1]))


entanglers = get_QCC_entanglers(ranked_entangler_groupings, n_ents, 2*lih.n_orbitals)

print('\nSelected entanglers:')
for ent in entanglers:
    print(ent)




Hamiltonian has 631 terms

Grouping gradient magnitudes (Grouping : Gradient magnitude):
1 : 0.123
2 : 0.0335
3 : 0.0335
4 : 0.024
5 : 0.024
6 : 0.0215
7 : 0.0121
8 : 0.0098
9 : 0.0098
10 : 0.0076
11 : 0.0076
12 : 0.0076
13 : 0.0076
14 : 0.0072
15 : 0.002
16 : 0.002
17 : 0.0013
18 : 0.0013
19 : 0.0012
20 : 0.0012
21 : 0.0007
22 : 0.0007
23 : 0.0004
24 : 0.0004
25 : 0.0002
26 : 0.0002
27 : 0.0
28 : 0.0
29 : 0.0
30 : 0.0
31 : 0.0
32 : 0.0

Selected entanglers:
1.0 [X2 Y3 X10 X11]


Once the QCC ranking procedure has been ran, we can simulate the QCC VQE optimization with the generated entanglers. The VQE optimization for the QCC ansatz is of the form
$$E = \min_{\boldsymbol{\Omega}, \boldsymbol{\tau}} \langle \boldsymbol{\Omega} | U_{\text{ENT}}^\dagger (\boldsymbol{\tau}) \hat H  U_{\text{ENT}} (\boldsymbol{\tau}) | \boldsymbol{\Omega} \rangle $$
where $\boldsymbol{\Omega}$ denote collective Euler angles parameterizing single-qubit rotations, and $\boldsymbol{\tau}$ are entangler amplitudes. 

In [27]:
#Mean-field part of U (Omega):    
U_MF = construct_QMF_ansatz(n_qubits = 2*lih.n_orbitals)
#Entangling part of U:
U_ENT = construct_QCC_ansatz(entanglers)

U_QCC = U_MF + U_ENT

E = tq.ExpectationValue(H=H, U=U_QCC)

initial_vals = init_qcc_params(hf_reference, E.extract_variables())

#Minimize wrt the entangler amplitude and MF angles:
result = tq.minimize(objective=E, method="BFGS", initial_values=initial_vals, tol=1.e-6)

print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result.energy))

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 50 expectationvalues

active variables : 25

E=-6.25839976  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.141592653589793, gamma_3: 0.0, beta_4: 0.0, gamma_4: 0.0, beta_5: 0.0, gamma_5: 0.0, beta_6: 0.0, gamma_6: 0.0, beta_7: 0.0, gamma_7: 0.0, beta_8: 0.0, gamma_8: 0.0, beta_9: 0.0, gamma_9: 0.0, beta_10: 3.141592653589793, gamma_10: 0.0, beta_11: 3.141592653589793, gamma_11: 0.0, tau_0: 0.0}  samples= None
Optimization terminated successfully.
         Current function value: -6.258400
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1

Obtained QCC energy (1 entanglers): -6.258399755644741


We see that the QCC energy converged to the FCI energy with only a single entangler! 

### H4 in 6-31G basis

Let us move on to the problem of H$_4$O in 6-31G basis using the same active space as the UCCSD example.

In [3]:
xyz_data = get_molecular_data('h4', geometry=90, xyz_format=True)

basis = '6-31g'
active = { 'AG':[0,1], 'B1U': [0], 'B2U': [0]}
h4= tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set = basis, active_orbitals = active)


There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.


- UCCSD

In [4]:
H = h4.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

U_UCCSD = h4.make_uccsd_ansatz(initial_amplitudes='MP2',threshold=threshold, trotter_steps=trotter_steps)

E = tq.ExpectationValue(H=H, U=U_UCCSD)

print('\nNumber of UCCSD amplitudes: {} \n'.format(len(E.extract_variables())))

print('\nStarting optimization:\n')

result = tq.minimize(objective=E,  initial_values={k:0.0 for k in E.extract_variables()}, tol=1e-4, method='bfgs')



Hamiltonian has 105 terms


Number of UCCSD amplitudes: 4 


Starting optimization:

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 128 expectationvalues

active variables : 4

E=-1.93303733  angles= {(3, 0, 3, 0): 0.0, (2, 1, 2, 1): 0.0, (3, 1, 3, 1): 0.0, (2, 0, 2, 0): 0.0}  samples= None
E=-1.92926868  angles= {(3, 0, 3, 0): -0.0867493599653244, (2, 1, 2, 1): -0.09617340564727783, (3, 1, 3, 1): -0.06707680225372314, (2, 0, 2, 0): -0.20908880233764648}  samples= None
E=-1.94840960  angles= {(3, 0, 3, 0): -0.04088724847850404, (2, 1, 2, 1): -0.04532904836757326, (3, 1, 3, 1): -0.031615056087880414, (2, 0, 2, 0): -0.09854903619657156}  samples= None
E=-1.93634995  angles= {(3, 0, 3, 0): 0.04710623158006605, (2, 1, 2, 1): -0.15248204298221218, (3, 1, 3, 1): 0.003034757883097469, (2, 0, 2, 0

- QCC

In [6]:
hf_reference = hf_occ(2*h4.n_orbitals, h4.n_electrons)


H = h4.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

#Define number of entanglers to enter ansatz
n_ents = 4

#Rank entanglers using energy gradient criterion
ranked_entangler_groupings = generate_QCC_gradient_groupings(H.to_openfermion(), 
                                                             2*h4.n_orbitals,
                                                             hf_reference, 
                                                             cutoff=threshold)

print('Grouping gradient magnitudes (Grouping : Gradient magnitude):')
for i in range(len(ranked_entangler_groupings)):
    print('{} : {}'.format(i+1,ranked_entangler_groupings[i][1]))

entanglers = get_QCC_entanglers(ranked_entangler_groupings, n_ents, 2*h4.n_orbitals)

print('\nSelected entanglers:')
for ent in entanglers:
    print(ent)


Hamiltonian has 105 terms

Grouping gradient magnitudes (Grouping : Gradient magnitude):
1 : 0.1045
2 : 0.0481
3 : 0.0434
4 : 0.0335
5 : 0.0
6 : 0.0

Selected entanglers:
1.0 [X0 Y1 X4 X5]
1.0 [X2 Y3 X4 X5]
1.0 [X0 Y1 X6 X7]
1.0 [X2 Y3 X6 X7]


With the 4 chosen entanglers, we can then run the QCC VQE simulation.

In [7]:
#Mean-field part of U (Omega):    
U_MF = construct_QMF_ansatz(n_qubits = 2*h4.n_orbitals)
#Entangling part of U:
U_ENT = construct_QCC_ansatz(entanglers)

U_QCC = U_MF + U_ENT

E = tq.ExpectationValue(H=H, U=U_QCC)

initial_vals = init_qcc_params(hf_reference, E.extract_variables())

#Minimize wrt the entangler amplitude and MF angles:
result = tq.minimize(objective=E, method="BFGS", initial_values=initial_vals, tol=1.e-4, backend='qulacs')


print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result.energy))

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 40 expectationvalues

active variables : 20

E=-1.93303733  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.141592653589793, gamma_3: 0.0, beta_4: 0.0, gamma_4: 0.0, beta_5: 0.0, gamma_5: 0.0, beta_6: 0.0, gamma_6: 0.0, beta_7: 0.0, gamma_7: 0.0, tau_0: 0.0, tau_1: 0.0, tau_2: 0.0, tau_3: 0.0}  samples= None
E=-1.94498346  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.141592653589793, gamma_3: 0.0, beta_4: 0.0, gamma_4: 0.0, beta_5: 0.0, gamma_5: 0.0, beta_6: 0.0, gamma_6: 0.0, beta_7: 0.0, gamma_7: 0.0, tau_0: 0.10454440116882324, tau_1: 0.04808670282363891

We obtain chemical accuracy for H4 near equilibrium geometry with only 4 entanglers. The obtained energy is not as accurate as that of UCCSD for this problem, however the QCC optimization may be performed at a fraction of the UCCSD circuit depth. One can also increase the number of entanglers entering the QCC ansatz to increase accuracy. As a final check, one can always run $n$ VQE trials with random initial guesses to test if the optimization fell into a local minimum. ***(Warning: Completing n=10 trials may take a few minutes for this VQE instance).***

In [34]:
n = 10
result = minimize_E_random_guesses(objective=E, method='BFGS', tol=1e-4, n=n)

print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result))

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 40 expectationvalues

active variables : 20

E=+0.09397890  angles= {beta_0: 0.32002472418022987, gamma_0: 0.9789446630060965, beta_1: 5.173989667312738, gamma_1: 5.562589331243322, beta_2: 1.0646754114842814, gamma_2: 0.02037339454177548, beta_3: 3.196879193316352, gamma_3: 1.4026700095589146, beta_4: 5.078835380492168, gamma_4: 3.0694755850720927, beta_5: 3.846695072100676, gamma_5: 5.263406516129091, beta_6: 4.94367105175321, gamma_6: 1.2305530453021758, beta_7: 0.26769436783199535, gamma_7: 1.539898614280836, tau_0: 0.2971136721175308, tau_1: 3.2429087324173547, tau_2: 5.844722146067363, tau_3: 0.926303161330132}  samples= None
E=-0.27883215  angles= {beta_0: 0.44206791656685707, gamma_0: 0.9670504220307166, beta_1: 4.79171813906872, gamma_

### LiH using 6-31G
Here we use 6-31G basis to calculate E using FCI/UCCSD/QCC
- FCI

In [8]:
xyz_data = get_molecular_data('lih', geometry=1.5, xyz_format=True)
basis='6-31g'
active = {'A1': [0, 1], 'B1': [0], 'B2': [0]}
lih = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis, active_orbitals=active)

E_FCI = lih.compute_energy(method='fci', backend='qulacs')

print('FCI energy: {}'.format(E_FCI))


There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.
There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for fci.
FCI energy: -7.977129915419726


- UCCSD

In [9]:
H = lih.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

U_UCCSD = lih.make_uccsd_ansatz(initial_amplitudes='MP2',threshold=threshold, trotter_steps=trotter_steps)

E = tq.ExpectationValue(H=H, U=U_UCCSD)

print('\nNumber of UCCSD amplitudes: {} \n'.format(len(E.extract_variables())))

print('\nStarting optimization:\n')

result = tq.minimize(objective=E, initial_values={k:0.0 for k in E.extract_variables()}, tol=1e-6, backend='qulacs', method='bfgs')
print('\nObtained UCCSD energy: {}'.format(result.energy))



Hamiltonian has 105 terms


Number of UCCSD amplitudes: 8 


Starting optimization:

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 256 expectationvalues

active variables : 8

E=-7.97685913  angles= {(3, 0, 3, 0): 0.0, (2, 1, 2, 1): 0.0, (3, 1, 3, 1): 0.0, (2, 0, 2, 1): 0.0, (3, 0, 3, 1): 0.0, (2, 1, 2, 0): 0.0, (3, 1, 3, 0): 0.0, (2, 0, 2, 0): 0.0}  samples= None
E=-7.97706708  angles= {(3, 0, 3, 0): -0.0006241798400878906, (2, 1, 2, 1): -0.01934528350830078, (3, 1, 3, 1): -0.01934528350830078, (2, 0, 2, 1): 0.0015673637390136719, (3, 0, 3, 1): 0.0015673637390136719, (2, 1, 2, 0): 0.0015673637390136719, (3, 1, 3, 0): 0.0015673637390136719, (2, 0, 2, 0): -0.0006241798400878906}  samples= None
E=-7.97682716  angles= {(3, 0, 3, 0): 0.0039079496940328425, (2, 1, 2, 1): -0.017093156109609754,

KeyboardInterrupt: 

- QCC

In [10]:
hf_reference = hf_occ(2*lih.n_orbitals, lih.n_electrons)

H = lih.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))


#Define number of entanglers to enter ansatz
n_ents = 2

#Rank entanglers using energy gradient criterion
ranked_entangler_groupings = generate_QCC_gradient_groupings(H.to_openfermion(),
                                                             2*lih.n_orbitals,
                                                             hf_reference,
                                                             cutoff=threshold)

print('Grouping gradient magnitudes (Grouping : Gradient magnitude):')
for i in range(len(ranked_entangler_groupings)):
    print('{} : {}'.format(i+1,ranked_entangler_groupings[i][1]))


entanglers = get_QCC_entanglers(ranked_entangler_groupings, n_ents, 2*lih.n_orbitals)

print('\nSelected entanglers:')
for ent in entanglers:
    print(ent)


Hamiltonian has 105 terms

Grouping gradient magnitudes (Grouping : Gradient magnitude):
1 : 0.0097
2 : 0.0097
3 : 0.0008
4 : 0.0008
5 : 0.0008
6 : 0.0008
7 : 0.0003
8 : 0.0003

Selected entanglers:
1.0 [X2 Y3 X4 X5]
1.0 [X2 Y3 X6 X7]


In [11]:
#Mean-field part of U (Omega):
U_MF = construct_QMF_ansatz(n_qubits = 2*lih.n_orbitals)
#Entangling part of U:
U_ENT = construct_QCC_ansatz(entanglers)

U_QCC = U_MF + U_ENT

E = tq.ExpectationValue(H=H, U=U_QCC)

initial_vals = init_qcc_params(hf_reference, E.extract_variables())

#Minimize wrt the entangler amplitude and MF angles:
result = tq.minimize(objective=E, method="BFGS", initial_values=initial_vals, tol=1.e-4, backend='qulacs')


print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result.energy))


Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 36 expectationvalues

active variables : 18

E=-7.97685913  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.141592653589793, gamma_3: 0.0, beta_4: 0.0, gamma_4: 0.0, beta_5: 0.0, gamma_5: 0.0, beta_6: 0.0, gamma_6: 0.0, beta_7: 0.0, gamma_7: 0.0, tau_0: 0.0, tau_1: 0.0}  samples= None
E=-7.97701384  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 3.141592415171214, gamma_2: 0.0, beta_3: 3.141592415171214, gamma_3: 0.0, beta_4: 0.0, gamma_4: 0.0, beta_5: 0.0, gamma_5: 0.0, beta_6: 0.0, gamma_6: 0.0, beta_7: 0.0, gamma_7: 0.0, tau_0: 0.00967264175415039, tau_1: 0.00967264175415039}  samples= None
E=-7.97

## NH3

- FCI w/ STO-3G

In [12]:

xyz_data = get_molecular_data('nh3', geometry=1.5, xyz_format=True)
basis='sto-3g'

nh3 = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis)

print('Number of spin-orbitals (qubits): {} \n'.format(2*nh3.n_orbitals))

E_FCI = nh3.compute_energy(method='fci', backend='qulacs')

print('FCI energy: {}'.format(E_FCI))

Number of spin-orbitals (qubits): 16 

FCI energy: -55.32940782498641


- FCI w/6-31G

In [13]:
basis='6-31g'

nh3 = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis)


print(list(map(lambda item: item[1][0], nh3.orbitals_by_irrep.items())))
print(list(map(lambda item: item[1][0].energy, nh3.orbitals_by_irrep.items())))

[QuantumChemistryPsi4.OrbitalData(irrep="A'", idx_irrep=0, idx_total=0, energy=-15.682567409874116), QuantumChemistryPsi4.OrbitalData(irrep='A"', idx_irrep=0, idx_total=2, energy=-0.47571264826135573)]
[-15.682567409874116, -0.47571264826135573]


In [14]:
active = {'A\'': [2, 3], 'A"': [0, 1]}
nh3 = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis, active_orbitals=active)

print('Number of spin-orbitals (qubits): {} \n'.format(2*nh3.n_orbitals))

E_FCI = nh3.compute_energy(method='fci', backend='qulacs')

print('FCI energy: {}'.format(E_FCI))

There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.
Number of spin-orbitals (qubits): 8 

There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for fci.
FCI energy: -55.91582659486136


- UCCSD w/6-31G


In [18]:
H = nh3.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

U_UCCSD = nh3.make_uccsd_ansatz(initial_amplitudes='MP2',threshold=threshold, trotter_steps=trotter_steps)

E = tq.ExpectationValue(H=H, U=U_UCCSD)

print('\nNumber of UCCSD amplitudes: {} \n'.format(len(E.extract_variables())))

print('\nStarting optimization:\n')

result = tq.minimize(objective=E,  initial_values={k:0.0 for k in E.extract_variables()}, tol=1e-4, method='bfgs')



Hamiltonian has 173 terms


Number of UCCSD amplitudes: 5 


Starting optimization:

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 160 expectationvalues

active variables : 5

E=-55.89755128  angles= {(3, 0, 3, 0): 0.0, (3, 2, 3, 1): 0.0, (3, 1, 3, 1): 0.0, (3, 1, 3, 2): 0.0, (3, 2, 3, 2): 0.0}  samples= None
E=-55.91076011  angles= {(3, 0, 3, 0): -0.21731948852539062, (3, 2, 3, 1): -0.030841827392578125, (3, 1, 3, 1): -0.04653167724609375, (3, 1, 3, 2): -0.030841827392578125, (3, 2, 3, 2): -0.061626434326171875}  samples= None
E=-55.91581393  angles= {(3, 0, 3, 0): -0.14209939410142164, (3, 2, 3, 1): -0.024074677804474108, (3, 1, 3, 1): -0.02166660543360647, (3, 1, 3, 2): -0.02407467780447411, (3, 2, 3, 2): -0.04620084889566002}  samples= None
E=-55.91581903  angles= {(3, 0, 3, 0): -0.14

- QCC w/6-31G

In [15]:
hf_reference = hf_occ(2*nh3.n_orbitals, nh3.n_electrons)

H = nh3.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))


Hamiltonian has 173 terms



In [16]:
#Define number of entanglers to enter ansatz
n_ents = 5

#Rank entanglers using energy gradient criterion
ranked_entangler_groupings = generate_QCC_gradient_groupings(H.to_openfermion(),
                                                             2*nh3.n_orbitals,
                                                             hf_reference,
                                                             cutoff=threshold)

print('Grouping gradient magnitudes (Grouping : Gradient magnitude):')
for i in range(len(ranked_entangler_groupings)):
    print('{} : {}'.format(i+1,ranked_entangler_groupings[i][1]))


entanglers = get_QCC_entanglers(ranked_entangler_groupings, n_ents, 2*nh3.n_orbitals)

print('\nSelected entanglers:')
for ent in entanglers:
    print(ent)

U_MF = construct_QMF_ansatz(n_qubits = 2*nh3.n_orbitals)
#Entangling part of U:
U_ENT = construct_QCC_ansatz(entanglers)

U_QCC = U_MF + U_ENT

E = tq.ExpectationValue(H=H, U=U_QCC)

initial_vals = init_qcc_params(hf_reference, E.extract_variables())

#Minimize wrt the entangler amplitude and MF angles:
result = tq.minimize(objective=E, method="BFGS", initial_values=initial_vals, tol=1.e-4, backend='qulacs')


print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result.energy))

Grouping gradient magnitudes (Grouping : Gradient magnitude):
1 : 0.1087
2 : 0.0308
3 : 0.0233
4 : 0.0154
5 : 0.0154
6 : 0.0
7 : 0.0

Selected entanglers:
1.0 [X0 Y1 X6 X7]
1.0 [X4 Y5 X6 X7]
1.0 [X2 Y3 X6 X7]
1.0 [X2 Y5 X6 X7]
1.0 [X3 Y4 X6 X7]
Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 42 expectationvalues

active variables : 21

E=-55.89755128  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.141592653589793, gamma_3: 0.0, beta_4: 3.141592653589793, gamma_4: 0.0, beta_5: 3.141592653589793, gamma_5: 0.0, beta_6: 0.0, gamma_6: 0.0, beta_7: 0.0, gamma_7: 0.0, tau_0: 0.0, tau_1: 0.0, tau_2: 0.0, tau_3: 0.0, tau_4: 0.0}  samples= None
E=-55.90874580  angles= {beta_0: 3.141592653589793, gamma_0: 0.0

## N2

- FCI w/STO-3G

In [1]:
import numpy as np
import tequila as tq
from utility import *
threshold = 1e-6 #Cutoff for UCC MP2 amplitudes and QCC ranking gradients
trotter_steps = 1


xyz_data = get_molecular_data('n2', geometry=1.5, xyz_format=True)
basis='sto-3g'

n2 = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis)

print('Number of spin-orbitals (qubits): {} \n'.format(2*n2.n_orbitals))

E_FCI = n2.compute_energy(method='fci', backend='qulacs')

print('FCI energy: {}'.format(E_FCI))


Number of spin-orbitals (qubits): 20 

FCI energy: -107.58163492166018


- FCI w/6-31G

In [2]:
basis='6-31g'

n2 = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis)
print(list(map(lambda item: item[1][0], n2.orbitals_by_irrep.items())))
print(list(map(lambda item: item[1][0].energy, n2.orbitals_by_irrep.items())))

[QuantumChemistryPsi4.OrbitalData(irrep='AG', idx_irrep=0, idx_total=0, energy=-15.806613506138117), QuantumChemistryPsi4.OrbitalData(irrep='B1U', idx_irrep=0, idx_total=1, energy=-15.805900361433766), QuantumChemistryPsi4.OrbitalData(irrep='B2U', idx_irrep=0, idx_total=5, energy=-0.4968324586317101), QuantumChemistryPsi4.OrbitalData(irrep='B3U', idx_irrep=0, idx_total=6, energy=-0.49683245863119674), QuantumChemistryPsi4.OrbitalData(irrep='B3G', idx_irrep=0, idx_total=7, energy=-0.005793047017178504), QuantumChemistryPsi4.OrbitalData(irrep='B2G', idx_irrep=0, idx_total=8, energy=-0.005793047016557085)]
[-15.806613506138117, -15.805900361433766, -0.4968324586317101, -0.49683245863119674, -0.005793047017178504, -0.005793047016557085]


In [3]:
active = {'AG': [1, 2], 'B1U': [1], 'B2U': [1]}
n2 = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis, active_orbitals=active)

print('Number of spin-orbitals (qubits): {} \n'.format(2*n2.n_orbitals))

E_FCI = n2.compute_energy(method='fci', backend='qulacs')

print('FCI energy: {}'.format(E_FCI))

There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.
Number of spin-orbitals (qubits): 8 

There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for fci.
FCI energy: -108.62442717371962


- UCCSD w/6-31G

In [4]:
H = n2.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

U_UCCSD = n2.make_uccsd_ansatz(initial_amplitudes='MP2',threshold=threshold, trotter_steps=trotter_steps)

E = tq.ExpectationValue(H=H, U=U_UCCSD)

print('\nNumber of UCCSD amplitudes: {} \n'.format(len(E.extract_variables())))

print('\nStarting optimization:\n')

result = tq.minimize(objective=E,  initial_values={k:0.0 for k in E.extract_variables()}, tol=1e-4, method='bfgs')



Hamiltonian has 105 terms


Number of UCCSD amplitudes: 5 


Starting optimization:

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 160 expectationvalues

active variables : 5

E=-108.62411657  angles= {(3, 0, 3, 0): 0.0, (3, 1, 3, 1): 0.0, (3, 2, 3, 0): 0.0, (3, 0, 3, 2): 0.0, (3, 2, 3, 2): 0.0}  samples= None
E=-108.61512395  angles= {(3, 0, 3, 0): -0.03958892822265625, (3, 1, 3, 1): -0.04364013671875, (3, 2, 3, 0): -0.00821685791015625, (3, 0, 3, 2): -0.00821685791015625, (3, 2, 3, 2): -0.01963043212890625}  samples= None
E=-108.62442311  angles= {(3, 0, 3, 0): -0.0060732169145442565, (3, 1, 3, 1): -0.0066947004723825684, (3, 2, 3, 0): -0.0012605231483839208, (3, 0, 3, 2): -0.0012605231483839208, (3, 2, 3, 2): -0.0030114448103916694}  samples= None
E=-108.62433384  angles= {(3, 0, 3, 0)

- QCC w/6-31G

In [5]:
hf_reference = hf_occ(2*n2.n_orbitals, n2.n_electrons)

H = n2.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))


Hamiltonian has 105 terms



In [7]:
#Define number of entanglers to enter ansatz
n_ents = 5

#Rank entanglers using energy gradient criterion
ranked_entangler_groupings = generate_QCC_gradient_groupings(H.to_openfermion(),
                                                             2*n2.n_orbitals,
                                                             hf_reference,
                                                             cutoff=threshold)

print('Grouping gradient magnitudes (Grouping : Gradient magnitude):')
for i in range(len(ranked_entangler_groupings)):
    print('{} : {}'.format(i+1,ranked_entangler_groupings[i][1]))

Grouping gradient magnitudes (Grouping : Gradient magnitude):
1 : 0.0218
2 : 0.0198
3 : 0.0098
4 : 0.0041
5 : 0.0041


In [8]:
entanglers = get_QCC_entanglers(ranked_entangler_groupings, n_ents, 2*n2.n_orbitals)

print('\nSelected entanglers:')
for ent in entanglers:
    print(ent)

U_MF = construct_QMF_ansatz(n_qubits = 2*n2.n_orbitals)
#Entangling part of U:
U_ENT = construct_QCC_ansatz(entanglers)

U_QCC = U_MF + U_ENT

E = tq.ExpectationValue(H=H, U=U_QCC)

initial_vals = init_qcc_params(hf_reference, E.extract_variables())

#Minimize wrt the entangler amplitude and MF angles:
result = tq.minimize(objective=E, method="BFGS", initial_values=initial_vals, tol=1.e-4, backend='qulacs')


print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result.energy))


Selected entanglers:
1.0 [X2 Y3 X6 X7]
1.0 [X0 Y1 X6 X7]
1.0 [X4 Y5 X6 X7]
1.0 [X0 Y5 X6 X7]
1.0 [X1 Y4 X6 X7]
Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 42 expectationvalues

active variables : 21

E=-108.62411657  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.141592653589793, gamma_3: 0.0, beta_4: 3.141592653589793, gamma_4: 0.0, beta_5: 3.141592653589793, gamma_5: 0.0, beta_6: 0.0, gamma_6: 0.0, beta_7: 0.0, gamma_7: 0.0, tau_0: 0.0, tau_1: 0.0, tau_2: 0.0, tau_3: 0.0, tau_4: 0.0}  samples= None
E=-108.62430214  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.141592653589793, gamma_3: 0.0, beta_4